# Install Libraries
1. Install the Google APIs Client Library for Python:
```pip install --upgrade google-api-python-client```

2. Install the `google-auth-oauthlib` and `google-auth-httplib2` libraries for user authorization.
```pip install --upgrade google-auth-oauthlib google-auth-httplib2```

# Setup Project and Credentials
1. In the [https://console.developers.google.com/](API Console), search for the YouTube Data API v3 and **enable** it.
2. Click on "Create Credentials" from the overview panel to go to credentials panel 
    1. Select the Youtube v3 API from dropdown
    2. Create an API key for public data that does not require user authorization (e.g. to retrieve information about a public YouTube channel). You can restrict it to Youtube API only from the Credentials Panel
    3. Create an OAuth 2.0 client ID for requests that need user authorization (currently authenticated user's YT)
        1. Set application type to desktop
    4. Download the JSON file that contains the OAuth 2.0 credentials. The file has a name like `client_secret_CLIENTID.json`, where CLIENTID is the client ID for your project.

In [1]:
# Read the API Key
with open("yt_api_key.txt", "r") as api_file:
    api_key = api_file.read().strip()


# Building and calling the Youtube Service

## Build the service object
- Use the `build()` function to create a service object with methods specific to given API. Arguments
    - API name
    - API version. 
    - `cache_discovery`: If `True`, artifact will be legacy cache (oauth2client<4.0). If not available in the legacy cache,  it will use the service definition shipped in the library. 
    - `static_discovery=False`: will retrieve the latest version of service definition from the internet (default is `True`).
- (List of all API versions are on the Supported APIs page)[https://github.com/googleapis/google-api-python-client/blob/main/docs/dyn/index.md]
- `httplib2` makes all connections persistent by default. Use the service object with a context manager or call close to avoid leaving sockets open.

```python
# Method 1
from googleapiclient.discovery import build

service = build('drive', 'v3')
# ...
service.close()

# Method 2
from googleapiclient.discovery import build

with build('drive', 'v3') as service:
    # ...
```

In [15]:
# Creating the Youtube Client

service_name = 'youtube'
service_version = 'v3'

from googleapiclient.discovery import build
from apiclient.errors import HttpError

youtube = build(service_name, service_version, developerKey=api_key)


## Collections
- Each API service provides access to one or more resources. A set of resources of the same type is called a collection. 
- The names of collections are specific to the API. 
- The service object has a function for every collection defined by the API. 
- The collection object can be created by using the `service.<collection_function>()` function.
- Collection may have nested child collections.

## Methods and requests
- Every collection has a list of methods defined by the API. 
- Calling a collection's method returns an HttpRequest object, which can then be executed to get a json response
- E.g. 
```python

# Create a request object using collection method
request = collection.list(cents=5)

# Execute request to get response
try:
    response = request.execute()
except HttpError as e:
    print('Error response status code : {0}, reason : {1}'.format(e.status_code, e.error_details))

# Use Response JSON object
import json
print(json.dumps(response, sort_keys=True, indent=4))

```

## More Information
- [APIs Explorer](https://developers.google.com/apis-explorer/) to browse APIs, list available methods.
- [Googleapi Documentation](https://googleapis.github.io/google-api-python-client/docs/epy/index.html)
- [Library Reference Documentation by API](https://github.com/googleapis/google-api-python-client/blob/main/docs/dyn/index.md)
- [Youtube Data API Functions](https://googleapis.github.io/google-api-python-client/docs/dyn/youtube_v3.html)
- [REST API for Youtube](https://developers.google.com/youtube/v3/docs) is the reference for Youtube API methods

In [3]:
# Find coordinates of USA
import requests
url = "https://nominatim.openstreetmap.org/search"
params = {"country": "in", "format":"json"}
resp = requests.get(url, params).json()[0]
latitude = resp['lat']
longitude = resp['lon']
bound_box = resp['boundingbox']

print("India is lat: {}, longitude:{}, bounded as: {}".format(latitude, longitude, bound_box))


India is lat: 22.3511148, longitude:78.6677428, bounded as: ['6.5531169', '35.6745457', '67.9544415', '97.395561']


In [20]:
# Search for a term
import json

search_term = 'cooking channels usa'
countries_allowed = ['US']
max_channels = 100
channel_list = []
total_channels = 0
rejected_channels = 0
page_token = None

yt_search = youtube.search()
search_request = yt_search.list(
    part = 'snippet', maxResults=50, regionCode='US',
    q=search_term, type='channel', order='relevance',
    topicId='/m/02wbm'
)

while total_channels < max_channels and search_request is not None:
    
    try:
        response = search_request.execute()
    except HttpError as httpe:
        error_dict = json.loads(httpe.content.decode('utf-8'))
        print("Error ({}) in calling Youtube API: {}".format(
            httpe.resp.status, error_dict['error']['message']
        ))
        print("Totally {} channels retrieved".format(total_channels))
        break

    search_result = response['items']
    for channel in search_result:
        channel_info = {}
        if(channel['id']['kind'] == 'youtube#channel'):

            # Get channel specific information
            channel_id = channel['id'].get('channelId')

            try:
                per_channel_detail = youtube.channels().list(
                    part="contentOwnerDetails, id, statistics, snippet, brandingSettings",
                    id=channel_id).execute()['items'][0]
            except HttpError as httpe:
                error_dict = json.loads(httpe.content.decode('utf-8'))
                print("Error ({}) in calling Youtube API: {}".format(
                    httpe.resp.status, error_dict['error']['message']
                ))
                print("Totally {} channels retrieved".format(total_channels))
                break

            channel_country = per_channel_detail['snippet'].get('country')

            if channel_country in countries_allowed:
                total_channels = total_channels + 1
                channel_info['sr'] = total_channels
                channel_info['link'] =  \
                    "https://youtube.com/channel/{}".format(channel_id)
                channel_info['title'] = channel['snippet'].get('title')
                channel_info['description'] = channel['snippet'].get('description')
                channel_info['country'] = per_channel_detail['snippet'].get('country')
                channel_info['keywords'] = per_channel_detail['brandingSettings']['channel'].get('keywords')
                channel_info['video_count'] = int(per_channel_detail['statistics'].get('videoCount', -1))
                channel_info['view_count'] = int(per_channel_detail['statistics'].get('viewCount', -1))
                channel_info['subscriber_count'] = int(per_channel_detail['statistics'].get('subscriberCount', -1))


                #TBD: Get Email
            
                # Append channel to list
                channel_list.append(channel_info)
                print("Channel {} with {} views from {} added to list. Added {} channels in total"\
                    .format(channel_info['title'], channel_info['view_count'], channel_country, total_channels)
                )
            else:
                rejected_channels += 1
                print('Reject Channel: {} with Country {}. Total Rejected {}'\
                    .format(channel_id, channel_country, rejected_channels))

    # Get Next Page
    search_request = yt_search.list_next(search_request, response)
    print(len(channel_list), " items in the list so far!")

   


Error (403) in calling Youtube API: The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.
Totally 0 channels retrieved


In [5]:
# Close the object and sockets
youtube.close()

In [13]:
# Create a Dataframe
import pandas as pd
channels_df = pd.DataFrame(channel_list)
channels_df_sorted = channels_df.sort_values(by=['view_count'], ascending=False)

channels_df_sorted


,sr,link,title,description,country,keywords,video_count,view_count,subscriber_count
52,53,https://youtube.com/channel/UCkqaQoGJ2rAHhWk7g...,5-Minute Recipes,Kitchen life hacks and yummy recipes make your...,US,"""-minute recipes"" recipes food meal dishes coo...",2355,2508236259,9410000
31,32,https://youtube.com/channel/UCayskv9f7BBHFWDVi...,Funny Stop Motion Cooking,Welcome to Funny Stop Motion Cooking Channel! ...,US,"""life hacks"" ""how to make"" diy mrgear mrhacker...",109,1054077563,2780000
47,48,https://youtube.com/channel/UCRIZtPl9nb9RiXc9b...,Food Wishes,"Hello this is Chef John, and welcome to the Fo...",None,recipes video cooking food tips cook wine dinn...,1735,881619027,4040000
16,17,https://youtube.com/channel/UCMmLIP_QfsDu_bavp...,Stop Motion Cooking,Welcome to Stop Motion Cooking Channel! Have y...,US,"""Stop Motion Cooking"" cooking ""stop motion"" as...",174,396418833,3110000
18,19,https://youtube.com/channel/UCekQr9znsk2vWxBo3...,You Suck At Cooking,No bullshit. Just cooking. (except for all of ...,None,"cooking funny ""how to"" tutorial ""you suck at c...",139,306708975,2770000
...,...,...,...,...,...,...,...,...,...
24,25,https://youtube.com/channel/UC_blgwNAB7aB0Pbpf...,cooking channels,,None,None,1,5,1
5,6,https://youtube.com/channel/UCrfgag66nYTcRFMVA...,cooking channels,,None,None,4,2,3
32,33,https://youtube.com/channel/UC_4lo9lQSjRETEx2W...,villagers and cook channels eats,"Today is our villag,we cook seafood stuffed in...",None,None,2,2,0
11,12,https://youtube.com/channel/UCF0o8X4vaf9YOLYpp...,Cooking channels,Indian Cooking.,None,None,2,1,0
